### NEAR Intents Examples Notebook

In [3]:
# !pip install requests -q

In [6]:
import os
import time
import datetime as dt
from typing import Optional, Dict, Any
import requests

In [13]:
BASE_URL = "https://1click.chaindefuser.com"
JWT = ""
# JWT = os.getenv("ONECLICK_JWT")  # put your token in env to avoid fees

##### **Methods**

In [14]:
def _headers() -> Dict[str, str]:
    if not JWT:
        # You can still call the API, but without the JWT you may incur a small fee.
        # See docs for how to obtain a JWT.
        pass
    return {
        "Authorization": f"Bearer {JWT}" if JWT else "",
        "Accept": "*/*",
        "Content-Type": "application/json",
    }


In [15]:
def list_supported_tokens() -> Any:
    """GET /v0/tokens"""
    r = requests.get(f"{BASE_URL}/v0/tokens", headers=_headers(), timeout=30)
    r.raise_for_status()
    return r.json()

In [16]:
def request_quote(
    *,
    dry: bool,
    swap_type: str,             # "EXACT_INPUT" | "EXACT_OUTPUT" | "FLEX_INPUT"
    slippage_bps: int,          # e.g., 100 for 1%
    origin_asset: str,          # e.g., "nep141:arb-0x...omft.near"
    deposit_type: str,          # "ORIGIN_CHAIN" | "INTENTS"
    destination_asset: str,     # e.g., "nep141:sol-...omft.near"
    amount_wei: str,            # smallest units (string)
    refund_to: str,             # address for refunds
    refund_type: str,           # "ORIGIN_CHAIN" | "INTENTS"
    recipient: str,             # matches recipientType
    recipient_type: str,        # "DESTINATION_CHAIN" | "INTENTS"
    deadline_iso: Optional[str] = None,  # ISO8601; required when dry=False
    referral: Optional[str] = None,
    quote_wait_ms: int = 3000,
    app_fees: Optional[list] = None,
    virtual_chain_recipient: Optional[str] = None,
    virtual_chain_refund_recipient: Optional[str] = None,
) -> Any:
    """POST /v0/quote"""
    if not deadline_iso:
        # reasonable default: now + 90 minutes
        deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"

    payload = {
        "dry": dry,
        "swapType": swap_type,
        "slippageTolerance": slippage_bps,
        "originAsset": origin_asset,
        "depositType": deposit_type,
        "destinationAsset": destination_asset,
        "amount": amount_wei,
        "refundTo": refund_to,
        "refundType": refund_type,
        "recipient": recipient,
        "recipientType": recipient_type,
        "deadline": deadline_iso,
        "quoteWaitingTimeMs": quote_wait_ms,
    }
    if referral:
        payload["referral"] = referral
    if app_fees:
        payload["appFees"] = app_fees
    if virtual_chain_recipient:
        payload["virtualChainRecipient"] = virtual_chain_recipient
    if virtual_chain_refund_recipient:
        payload["virtualChainRefundRecipient"] = virtual_chain_refund_recipient

    r = requests.post(f"{BASE_URL}/v0/quote", headers=_headers(), json=payload, timeout=60)
    r.raise_for_status()
    return r.json()

In [17]:
def submit_deposit_tx(tx_hash: str, deposit_address: str) -> Any:
    """POST /v0/deposit/submit  (optional but speeds up processing)"""
    payload = {"txHash": tx_hash, "depositAddress": deposit_address}
    r = requests.post(f"{BASE_URL}/v0/deposit/submit", headers=_headers(), json=payload, timeout=30)
    r.raise_for_status()
    return r.json()

In [18]:
def get_status(deposit_address: str) -> Any:
    """GET /v0/status?depositAddress=..."""
    r = requests.get(f"{BASE_URL}/v0/status", headers=_headers(), params={"depositAddress": deposit_address}, timeout=30)
    r.raise_for_status()
    return r.json()

##### **Testing**

**1) Discover tokens**

In [20]:
tokens = list_supported_tokens()

In [23]:
len(list(tokens))

94

In [24]:
# tokens

In [25]:
print("Supported tokens (truncated):", tokens[:3])  # sample

Supported tokens (truncated): [{'assetId': 'nep141:wrap.near', 'decimals': 24, 'blockchain': 'near', 'symbol': 'wNEAR', 'price': 2.82, 'priceUpdatedAt': '2025-08-09T23:23:00.051Z', 'contractAddress': 'wrap.near'}, {'assetId': 'nep141:eth.bridge.near', 'decimals': 18, 'blockchain': 'near', 'symbol': 'ETH', 'price': 4257.59, 'priceUpdatedAt': '2025-08-09T23:23:00.051Z', 'contractAddress': 'eth.bridge.near'}, {'assetId': 'nep141:17208628f84f5d6ad33f0da3bbbeb27ffcb398eac501a31bd6ad2011e36133a1', 'decimals': 6, 'blockchain': 'near', 'symbol': 'USDC', 'price': 0.999744, 'priceUpdatedAt': '2025-08-09T23:23:00.051Z', 'contractAddress': '17208628f84f5d6ad33f0da3bbbeb27ffcb398eac501a31bd6ad2011e36133a1'}]


**2) DRY quote (preview pricing; no deposit address generated)**

| Parameter                             | Type   | Meaning                                                                                                                                                                             |
| ------------------------------------- | ------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **dry**                               | `bool` | If `True`, preview the quote without creating a deposit address. If `False`, creates a deposit address and locks route parameters.                                                  |
| **swap\_type**                        | `str`  | Trade type: `"EXACT_INPUT"` (you specify the exact input amount), `"EXACT_OUTPUT"` (you specify the exact output amount you want), or `"FLEX_INPUT"` (let the system adjust input). |
| **slippage\_bps**                     | `int`  | Slippage tolerance in **basis points** (bps). `100` = 1% max price movement allowed before failing/refunding.                                                                       |
| **origin\_asset**                     | `str`  | Asset you are sending from the origin chain, formatted as `nep141:<chain>-<address>.omft.near` or similar.                                                                          |
| **deposit\_type**                     | `str`  | How funds are deposited: `"ORIGIN_CHAIN"` (send directly on the origin chain) or `"INTENTS"` (fund via a virtual chain).                                                            |
| **destination\_asset**                | `str`  | Asset you want to receive on the destination chain (same format as `origin_asset`).                                                                                                 |
| **amount\_wei**                       | `str`  | Amount of the origin asset in its smallest unit (integer string, e.g., "1000000" for 1 USDC with 6 decimals).                                                                       |
| **refund\_to**                        | `str`  | Address that should receive refunds if the swap fails or times out.                                                                                                                 |
| **refund\_type**                      | `str`  | `"ORIGIN_CHAIN"` (refund on original chain) or `"INTENTS"` (refund via virtual chain).                                                                                              |
| **recipient**                         | `str`  | Address that will receive the destination asset. Must match `recipient_type` format.                                                                                                |
| **recipient\_type**                   | `str`  | `"DESTINATION_CHAIN"` (deliver directly on the destination chain) or `"INTENTS"` (deliver to a virtual chain).                                                                      |
| **deadline\_iso**                     | `str`  | ISO 8601 UTC timestamp for when the quote expires (e.g., `"2025-08-09T18:59:00Z"`). Required for live quotes.                                                                       |
| **referral**                          | `str`  | Optional referral code or app identifier for tracking.                                                                                                                              |
| **quote\_wait\_ms**                   | `int`  | Time to wait (ms) for route computation before returning (default `3000`).                                                                                                          |
| **app\_fees**                         | `list` | Optional per-swap fee list, each `{ "recipient": "<address>", "fee": <bps> }`.                                                                                                      |
| **virtual\_chain\_recipient**         | `str`  | (Optional) Address on a virtual chain to receive funds instead of destination chain.                                                                                                |
| **virtual\_chain\_refund\_recipient** | `str`  | (Optional) Address on a virtual chain for refunds instead of origin chain.                                                                                                          |


In [ ]:
dry_quote = request_quote(
    dry=True,  # preview the quote without creating a deposit address
    swap_type="EXACT_INPUT",  # specify the exact input amount
    slippage_bps=100,  # 1%
    origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",  # asset you are sending from the origin chain
    deposit_type="ORIGIN_CHAIN",  # how funds are deposited, send directly on the origin chain
    destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",  # Asset you want to receive on the destination chain (same format as origin chain)
    amount_wei="1000000",  # example: 1.0 token if decimals=6
    refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",  # Address that should receive refunds if the swap fails or times out.
    refund_type="ORIGIN_CHAIN",
    recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",  # Address that will receive the destination asset. Must match recipient_type format.
    recipient_type="DESTINATION_CHAIN",
    referral="myapp",
    quote_wait_ms=3000,
    app_fees=[{"recipient": "recipient.near", "fee": 100}],  # 100 bps = 1%
)
print("DRY quote:", dry_quote)

/tmp/ipykernel_164851/2870745585.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"


DRY quote: {'quote': {'amountIn': '1000000', 'amountInFormatted': '1.0', 'amountInUsd': '0.9998', 'minAmountIn': '1000000', 'amountOut': '592526', 'amountOutFormatted': '0.592526', 'amountOutUsd': '0.5924', 'minAmountOut': '582626', 'timeEstimate': 22}, 'quoteRequest': {'quoteWaitingTimeMs': 3000, 'dry': True, 'swapType': 'EXACT_INPUT', 'slippageTolerance': 100, 'originAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near', 'depositType': 'ORIGIN_CHAIN', 'destinationAsset': 'nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near', 'amount': '1000000', 'refundTo': '0x2527D02599Ba641c19FEa793cD0F167589a0f10D', 'refundType': 'ORIGIN_CHAIN', 'recipient': '13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK', 'recipientType': 'DESTINATION_CHAIN', 'deadline': '2025-08-10T01:11:51.000Z', 'referral': 'myapp', 'appFees': [{'recipient': 'recipient.near', 'fee': 100}]}, 'signature': 'ed25519:4MFrt6Jj12cidyZGk5qTMXty7nSeGQMtnzdsDKyQr4UR8mvgm8BGLpvQ4ZuZDrBBD5XiTJg4uuNxxhe4hF9zuZn

In [27]:
dry_quote

{'quote': {'amountIn': '1000000',
  'amountInFormatted': '1.0',
  'amountInUsd': '0.9998',
  'minAmountIn': '1000000',
  'amountOut': '592599',
  'amountOutFormatted': '0.592599',
  'amountOutUsd': '0.5925',
  'minAmountOut': '582699',
  'timeEstimate': 22},
 'quoteRequest': {'quoteWaitingTimeMs': 3000,
  'dry': True,
  'swapType': 'EXACT_INPUT',
  'slippageTolerance': 100,
  'originAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near',
  'depositType': 'ORIGIN_CHAIN',
  'destinationAsset': 'nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near',
  'amount': '1000000',
  'refundTo': '0x2527D02599Ba641c19FEa793cD0F167589a0f10D',
  'refundType': 'ORIGIN_CHAIN',
  'recipient': '13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK',
  'recipientType': 'DESTINATION_CHAIN',
  'deadline': '2025-08-10T00:54:27.000Z',
  'referral': 'myapp',
  'appFees': [{'recipient': 'recipient.near', 'fee': 100}]},
 'signature': 'ed25519:RNJwUriXorydLLdrQzWE17K6xLDYRq8RvhvCbqz4Z1gG8g8tDB7U

**3) Live quote (generates a unique deposit address you will fund)**

In [28]:
live_quote = request_quote(
    dry=False,
    swap_type="EXACT_INPUT",
    slippage_bps=100,
    origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
    deposit_type="ORIGIN_CHAIN",
    destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",
    amount_wei="1000000",
    refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",
    refund_type="ORIGIN_CHAIN",
    recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",
    recipient_type="DESTINATION_CHAIN",
    referral="myapp",
)

/tmp/ipykernel_164851/2870745585.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  deadline_iso = (dt.datetime.utcnow() + dt.timedelta(minutes=90)).replace(microsecond=0).isoformat() + "Z"


In [29]:
live_quote

{'quote': {'amountIn': '1000000',
  'amountInFormatted': '1.0',
  'amountInUsd': '0.9997',
  'minAmountIn': '1000000',
  'amountOut': '602503',
  'amountOutFormatted': '0.602503',
  'amountOutUsd': '0.6023',
  'minAmountOut': '592503',
  'timeEstimate': 22,
  'deadline': '2025-08-10T23:26:12.758Z',
  'timeWhenInactive': '2025-08-10T23:26:12.758Z',
  'depositAddress': '0x69f1e6F0140bd83263CE33111E85D0c975f8504B'},
 'quoteRequest': {'quoteWaitingTimeMs': 3000,
  'dry': False,
  'swapType': 'EXACT_INPUT',
  'slippageTolerance': 100,
  'originAsset': 'nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near',
  'depositType': 'ORIGIN_CHAIN',
  'destinationAsset': 'nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near',
  'amount': '1000000',
  'refundTo': '0x2527D02599Ba641c19FEa793cD0F167589a0f10D',
  'refundType': 'ORIGIN_CHAIN',
  'recipient': '13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK',
  'recipientType': 'DESTINATION_CHAIN',
  'deadline': '2025-08-10T00:56:08.000Z',


In [30]:
deposit_address = live_quote["quote"]["depositAddress"]
print("Deposit to:", deposit_address)

Deposit to: 0x69f1e6F0140bd83263CE33111E85D0c975f8504B


4. (You send tokens to deposit_address on the origin chain)

Optional: tell 1Click the tx hash immediately to speed up processing:

In [ ]:
submit_res = submit_deposit_tx(tx_hash="0x123abc456def789", deposit_address=deposit_address)

In [ ]:

# -------------------------
# Example usage (adjust as needed)
# -------------------------
if __name__ == "__main__":
    # 1) Discover tokens
    tokens = list_supported_tokens()
    print("Supported tokens (truncated):", tokens[:3])  # sample

    # 2) DRY quote (preview pricing; no deposit address generated)
    dry_quote = request_quote(
        dry=True,
        swap_type="EXACT_INPUT",
        slippage_bps=100,  # 1%
        origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
        deposit_type="ORIGIN_CHAIN",
        destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",
        amount_wei="1000000",  # example: 1.0 token if decimals=6
        refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",
        refund_type="ORIGIN_CHAIN",
        recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",
        recipient_type="DESTINATION_CHAIN",
        referral="myapp",
        quote_wait_ms=3000,
        app_fees=[{"recipient": "recipient.near", "fee": 100}],  # 100 bps = 1%
    )
    print("DRY quote:", dry_quote)

    # 3) Live quote (generates a unique deposit address you will fund)
    live_quote = request_quote(
        dry=False,
        swap_type="EXACT_INPUT",
        slippage_bps=100,
        origin_asset="nep141:arb-0xaf88d065e77c8cc2239327c5edb3a432268e5831.omft.near",
        deposit_type="ORIGIN_CHAIN",
        destination_asset="nep141:sol-5ce3bf3a31af18be40ba30f721101b4341690186.omft.near",
        amount_wei="1000000",
        refund_to="0x2527D02599Ba641c19FEa793cD0F167589a0f10D",
        refund_type="ORIGIN_CHAIN",
        recipient="13QkxhNMrTPxoCkRdYdJ65tFuwXPhL5gLS2Z5Nr6gjRK",
        recipient_type="DESTINATION_CHAIN",
        referral="myapp",
    )
    deposit_address = live_quote["quote"]["depositAddress"]
    print("Deposit to:", deposit_address)

    # 4) (You send tokens to deposit_address on the origin chain)
    # Optional: tell 1Click the tx hash immediately to speed up processing:
    # submit_res = submit_deposit_tx(tx_hash="0x123abc456def789", deposit_address=deposit_address)
    # print("Submitted deposit tx:", submit_res)

    # 5) Poll for status until success/refund
    for _ in range(30):
        status = get_status(deposit_address)
        print("Status:", status.get("status"))
        if status.get("status") in {"SUCCESS", "REFUNDED", "FAILED"}:
            break
        time.sleep(10)

In [ ]:
API_URL = "https://api.near-intents.io/v1/1click"
API_KEY = "YOUR_API_KEY"

In [5]:
payload = {
    "account_id": "example.near",
    "intent": {
        "receiver_id": "receiver.near",
        "actions": [
            {
                "type": "Transfer",
                "amount": "1000000000000000000000000"  # 1 NEAR in yoctoNEAR
            }
        ]
    }
}

In [ ]:
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
response = requests.post(API_URL, json=payload, headers=headers)

In [ ]:
print(response.status_code)
print(response.json())